In [15]:
import os 

In [16]:
%pwd

'/Users/sunilverma/Text-Summarizer-Project'

In [ ]:
#os.chdir("../")

In [ ]:
#%pwd

'/Users/sunilverma/Text-Summarizer-Project'

In [17]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: str
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: int
    gradient_accumulation_steps: int

In [18]:
from text_summarizer.constants import *
from text_summarizer.utils.common import read_yaml, create_directories

In [19]:
from pathlib import Path
from text_summarizer.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from text_summarizer.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
    ):
        # Convert relative paths to absolute
        if not config_filepath.is_absolute():
            config_filepath = Path(os.getcwd()) / config_filepath
        if not params_filepath.is_absolute():
            params_filepath = Path(os.getcwd()) / params_filepath
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        model_trainer_config = self.config.model_trainer
        training_args = self.params.TrainingArguments

        model_trainer_config = ModelTrainerConfig(
            root_dir=Path(model_trainer_config.root_dir),
            data_path=Path(model_trainer_config.data_path),
            model_ckpt=Path(model_trainer_config.model_ckpt),
            num_train_epochs=training_args.num_train_epochs,
            warmup_steps=training_args.warmup_steps,
            per_device_train_batch_size=training_args.per_device_train_batch_size,
            weight_decay=training_args.weight_decay,
            logging_steps=training_args.logging_steps,
            evaluation_strategy=training_args.evaluation_strategy,
            eval_steps=training_args.eval_steps,
            save_steps=training_args.save_steps,
            gradient_accumulation_steps=training_args.gradient_accumulation_steps,
        )
        return model_trainer_config

In [20]:
from transformers import TrainingArguments, Trainer 
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk 
import torch

In [11]:
# from transformers import TrainingArguments, Trainer

# trainer_args = TrainingArguments(
#     output_dir='pegasus-samsum', num_train_epochs=1, warmup_steps=500,
#     per_device_train_batch_size=1, per_device_eval_batch_size=1,
#     weight_decay=0.01, logging_steps=10,
#     eval_strategy='steps', eval_steps=500, save_steps=1e6,
#     gradient_accumulation_steps=16
# )

In [ ]:
class ModelTrainer: 
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        
        device = "cuda" if torch.cuda.is_available() else "cpu"
        print(f"Using device: {device}")
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt,use_safetensors=True)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

        data_collator = seq2seq_data_collator

        # Loading the dataset from the data path
        dataset_samsum_pt = load_from_disk(self.config.data_path)
        
        # Use only first 50 samples for faster training
        small_train_dataset = dataset_samsum_pt['test'].select(range(min(50, len(dataset_samsum_pt['test']))))
        print(f"Training on {len(small_train_dataset)} samples")

        # Direct insertion of values - optimized for speed
        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir,
            num_train_epochs=1,
            warmup_steps=50,  # Reduced from 500
            per_device_train_batch_size=2,  # Increased from 1
            weight_decay=0.01,
            logging_steps=5,  # Reduced from 10 for more frequent updates
            eval_strategy='no',  # Disabled evaluation to save time
            save_steps=int(1e6),  # No intermediate saves
            gradient_accumulation_steps=8  # Reduced from 16
        )

        trainer = Trainer(
            model=model,
            args=trainer_args,
            tokenizer=tokenizer,
            train_dataset=small_train_dataset,  # Using small subset
            data_collator=data_collator,
        )

        print("Starting training...")
        trainer.train()
        print("Training completed!")

        ## Saving the model after training
        model.save_pretrained(os.path.join(self.config.root_dir, "pegasus-samsum-model"))
        ## Save tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir, "tokenizer"))



In [28]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e: 
    raise e

2025-12-18 18:20:50,197: INFO: common: yaml file: /Users/sunilverma/Text-Summarizer-Project/config/config.yaml loaded successfully
2025-12-18 18:20:50,207: INFO: common: yaml file: /Users/sunilverma/Text-Summarizer-Project/params.yaml loaded successfully
2025-12-18 18:20:50,209: INFO: common: created directory at: artifacts
Using device: cpu


'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: d22846f6-1c60-41b7-8383-d86b3dfefe48)')' thrown while requesting HEAD https://huggingface.co/google/pegasus-cnn_dailymail/resolve/main/tokenizer_config.json


2025-12-18 18:21:00,254: WARNING: _http: '(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: d22846f6-1c60-41b7-8383-d86b3dfefe48)')' thrown while requesting HEAD https://huggingface.co/google/pegasus-cnn_dailymail/resolve/main/tokenizer_config.json


Retrying in 1s [Retry 1/5].


2025-12-18 18:21:00,256: WARNING: _http: Retrying in 1s [Retry 1/5].


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/var/folders/9m/ztkm5bns0x1b521x6wdpd4640000gn/T/ipykernel_2707/4232110294.py:32: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None}.


Starting training...


Step,Training Loss,Validation Loss


KeyboardInterrupt: 